```stata
*! NHANES Dataset Fast Creation
*! By Vincent Jin and Abimereki Muzaale

qui {
	
	capture program drop nhanes1
	program define nhanes1
	
		syntax , [ys(int 1999) ye(int 999999) ex(numlist) ds(int 0)]
		
		qui {
			
			if 1 { // basic checks
				
				// check ys
				if ((`ys' < 1988) | (inrange(`ys', 1995, 1998)) | (`ys' > 2020)) {
					noi di "************************ERROR***************************"
					noi di "*******No Data Available For Requested Start Year*******"
					noi di "********************************************************"
					assert
				} 
				
				// replace ye to default value
				if (`ye' == 999999) {
					local ye = `ys' + 1
				}
				
				// check if ye is greater than ys
				if !(`ye' > `ys') {
					noi di "************************ERROR***************************"
					noi di "******Requested End Year is Smaller Than Start Year*****"
					noi di "********************************************************"
					assert
				}
				
				// check ye
				if ((`ye' < 1988) | (inrange(`ye', 1996, 1999)) | (`ye' > 2021)) {
					noi di "************************ERROR***************************"
					noi di "*******No Data Available For Requested End Year*********"
					noi di "********************************************************"
					assert
				} 
				
				// modify ys for ys that starts in second year
				if (`ys' > 1998 & mod(`ys', 2) == 0) {
					local ys = `ys' - 1
					noi di "***********************WARNING**************************"
					noi di "*******Start Year Automatically Adjusted To `ys'********"
					noi di "*******Since Start Year Should Be An Uneven Year********"
					noi di "********************************************************"
				}
				
				// warning for NHANES III then modify ys to 1988
				if (inrange(`ys', 1988, 1994)) {
					noi di "***********************WARNING**************************"
					noi di "*****Data For Year Between 1988 To 1994 Requested*******"
					noi di "****NHANES III Data For That Period Will Be Loaded******"
					noi di "********************************************************"
					local ys = 1988
				}
				
				// warning for time gap
				if (`ys' < 1995 & `ye' > 1999) {
					noi di "***********************WARNING**************************"
					noi di "********No Data Between 1995 And 1999 Available*********"
					noi di "********************************************************"
				}
				
				// warning for pre-pandemic
				if ((`ys' < 2017 & `ye' > 2017) | inrange(`ys', 2017, 2020)) {
					noi di "***********************WARNING**************************"
					noi di "*****Data For Year Between 2017 To 2020 Requested*******"
					noi di "**NHANES Pre-Pandemic Data For That Period Will Be Load*"
					noi di "********************************************************"
				}
				
				// warning for 2020
				if (inrange(2020, `ys', `ye')) {
					noi di "***********************WARNING**************************"
					noi di "*****Only Data Until March Was Included For 2020********"
					noi di "********************************************************"
				}
				
				// current ds:
				/* 
				base          1
				demographic   2
				exam          3
				diet          4
				questionnaire 5
				
				*/
				local ds_helper base demographic exam diet questionnaire
				tokenize `ds_helper'
				local ds = `ds' + 1
				macro drop ds_helper2
				global ds_helper2 : di "``ds''"
				
			}
			
			if 2 { // print out summary for dataset creation and ask for confirmation
				
				// also prepare a list for exclusion to be tested
				// screen if ex_helper contains even years
				macro drop ex_helper2
				foreach i in `ex' {
					local disp_helper : di "`disp_helper'" " " `i'
					if (`i' > 1998 & mod(`i', 2) == 0) {
						local ex_helper = `i' - 1
					}
					else {
						local ex_helper = `i'
					}
					global ex_helper2 : di "`ex_helper'" ", " "${ex_helper2}"
				}
				global ex_helper2 : di substr("${ex_helper2}", 1, strlen("${ex_helper2}") - 2)

				noi di "NHANES Dataset Creation Summary"
				noi di "Requested Start Year: `ys'"
				noi di "Requested End Year: `ye'"
				if ("`disp_helper'" != "") {
					noi di "Requested Year To Be Excluded: `disp_helper'"
				}
				else {
					noi di "Requested Year To Be Excluded: None"
				}
				noi di "Requested Dataset Information: ${ds_helper2}"
				noi di "Current Saving Directory: `c(pwd)'"
				noi di "Please Confirm To Proceed (Y/N)", _request(confirmation)
				if !((strupper(substr("${confirmation}", 1, 1)) == "Y") | (substr("${confirmation}", 1, 1) == "1")) {
					noi di "User Requested To Abort Creation"
					assert
				}	
				
			}
			
			if 3 { // pulling dataset
				
				local ye_helper = `ye' - 1
				forvalues i = `ys'(1)`ye_helper' {
						if ("`ex'" != "") {
							if !(inlist(`i', ${ex_helper2})) {
								local year_helper `i' `year_helper'
							}
						}
						else {
							local year_helper `i' `year_helper'
						}
				}

				foreach i in `year_helper' {
					if (`i' > 1998 & mod(`i', 2) != 0) {
						local ys_2 `i' `ys_2'
					}
					else if (`i' < 1995 & `i' > 1987) {
						local ys_1 `i' `ys_1'
					}
				}

				local dataset_counter = 0

				// loading the NHANES III Data
				if ("`ys_1'" != "") {
					
					local dataset_counter = `dataset_counter' + 1
					
					noi di "Loading NHANES III Data For 1988-1994...................", _continue
					
					macro drop nh3
					global nh3 https://wwwn.cdc.gov/nchs/data/nhanes3/1a/
					
					// base dataset
					*** PERHAPS CONDENSE DS CONDITIONS TO ONE MACRO ****
					*** WORK ON THIS LATER***
					if (`ds' == 1) {
						noi infix seqn 1-5 ///
									dmarethn 12 ///
									dmaracer 13 ///
									dmaethnr 14 ///
									hssex 15 ///
									hsageir 18-19 ///
									using ${nh3}adult.DAT, clear
					}
					
					// demographic dataset
					if (`ds' == 2) {
						
						noi infix seqn 1-5 ///
									dmpstat 11 /// 
									dmarethn 12 ///
									dmaethnr 14 ///
									hssex 15 ///
									hsageir	18-19 ///
									hfa8r 1256-1257 ///
									hff19r 1409-1410 ///
									sdpphase 42 ///
									using ${nh3}adult.DAT, clear
													
					}
					
					// save to a tempfile for future appending
					tempfile ds`dataset_counter'
					capture drop year_start
					gen year_start = 1988
					save ds`dataset_counter', replace

				}
				
				// loading NHANES Data in other years
				// having prepan_counter to ensure 2017-2020 only loaded once
				if ("`ys_2'" != "") {
					
					local prepan_counter = 0
					
					macro drop nhanes
					global nhanes "https://wwwn.cdc.gov/Nchs/Nhanes"
					
					foreach i of numlist `ys_2' {
						
						local ye_helper2 = `i' + 1
						
						local name_helper = `i' - 1999
						if (`name_helper' > 0) {
							local name_helper2 = 2 + ((`name_helper' - 2) / 2)
							tokenize "`c(ALPHA)'"
							local name_helper3 : di "_" "``name_helper2''"
							if (("`name_helper3'" == "_J") | ("`name_helper3'" == "_K")) {
								local name_helper3 : di "P_"
							}
						}

						if (!(inrange(`i', 2017, 2020)) | (inrange(`i', 2017, 2020) & `prepan_counter' == 0)) {
								
							local dataset_counter = `dataset_counter' + 1
							if (inrange(`i', 2017, 2020)) {
								local prepan_counter = `prepan_counter' + 1
							}
							
							if (`ds' == 2) {
								
								if !(inrange(`i', 2017, 2020)) {
									local ds_name : di "DEMO`name_helper3'"
									local period : di "`i'" "-" "`ye_helper2'"
								}
								else if (inrange(`i', 2017, 2020)) {
									local ds_name : di "`name_helper3'DEMO"
									local period : di "2017-2020"
								}
								noi di "Loading NHANES Data For " "`period'...................", _continue
								import sasxport5 "$nhanes/`i'-`ye_helper2'/`ds_name'.XPT", clear
								tempfile ds`dataset_counter'
								capture drop year_start
								gen year_start = `i'
								save ds`dataset_counter', replace
								noi di "(" "`c(N)'" " " "observations read)"
								
							}

						}
						
					}
					
				}
				
				local ds_max = `dataset_counter' - 1
				
				noi di "Appending All Datasets..............................", _continue
				forvalues i = 1/`ds_max' {
					
					append using ds`i'
					
				}
				
				compress
				noi di "Done"
				
				save NHANES_${ds_helper2}_`ys'_`ye', replace
				noi di "Dataset Saved!"
				
			}
			
		}
	end
	
}
```